<a href="https://colab.research.google.com/github/MattWroclaw/data-science-bootcamp/blob/main/Uczenie_maszynowe/metryki_klasyfikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## scikit learn
Biblioteka: https://scikit-learn.org/stable/  
User Guide: https://scikit-learn.org/stable/user_guide.html

### Klasyfikacja binarna
Gdzie mamy do przewidzenia 2 klasy: koty-psy, czy klient odejdzie..   
   1. Dokładoność klasyfikacji `Accuracy`   

   *Accuracy* =  (correct predictions/total predictions) *100

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
y_true=[1,0,1,1,0,1,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,1,1,0]
y_pred=[0,0,1,1,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,0,1,1,0,0,1,1,0,1,1,0]

In [3]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.7333333333333333

In [ ]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,0
6,1,0
7,0,1
8,0,0
9,1,1


In [ ]:
results = results.sort_values(by=['y_true'])
results = results.reset_index(drop=True)
results['sample'] = results.index + 1
results

,y_true,y_pred,sample
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,1,5
5,0,0,6
6,0,0,7
7,0,1,8
8,0,0,9
9,0,0,10


In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(height=600, width=800, title_text="Klasyfikacja binarna")
fig.show()

## Macierz konfuzji/pomyłek

In [5]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
cm

array([[10,  4],
       [ 4, 12]])

Interesuje nas maxymalizacja wartości na przekątnych

In [6]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
  cm = cm[::-1]  # zamiana kolejności macierzy
  cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=500, height=500, title_text="Macierz konfuzji", font_size=16)
  fig.show()

plot_confusion_matrix(cm)

Po lewej stronie jest są to wartości prawdziwe, to co jest na górze to jest przewidziane. Mamy klasę 0, model przewidział klasę 0 w 10 przypadkach, pomylił się w 4.  
Mamy klasę 1. Model przewidział popwrawnie klasę 1 w 12 przypadkach, ale pomylił sie w 4 przypadkach.
*Idealnie*
Przekątna LG->PD => max
Przekątna PG->LD => 0  

Zawsze rozpatruje się każdy kawałek oddzielnie, nie mozna patrzeć symetrycznie (przykład z terorsytą, lepiej jak normalnego weźmiemy za terrorystę niż terorystę na normalnego)

Klasa 0 *NEGATIVE*  
Klasa 1 *POSITIVE*

In [ ]:
# Rozpiszemy sobie true-negative, false-positve, false-negative, true-positive
# biorąc każdy el. macierzy konfuzji
# cm.ravel() ---> array([10,  4,  4, 12])
#  Przyjmujemy klasa negatywna -> klasa 0, klasa negatywna -> klasa 1
tn, fp, fn, tp = cm.ravel()
tn, fp, fn, tp
print(f'tn - true negative : {tn}')
print(f'fp - false positive : {fp}')
print(f'fn - false negative : {fn}')
print(f'tp - true positive: {tp}')



tn - true negative : 10
fp - false positive : 4
fn - false negative : 4
tp - true positive: 12


In [ ]:
# Błąd pierwszego rodzaju: Type I error - False Positive Rate
#  Błąd ten mowi jaki jest wskaźnik wartości włszywie pozytywnycch (u nas 4 , gr-prawa)
# u nas to jest 4 / (4+10)

fpr = fp / (fp + tn)
fpr

0.2857142857142857

In [ ]:
# Type II error - False Negative Error
fnr = fn / (fn + tp)
fnr
#  u nas to jest 4 / (4+12)
# ten błąd jest dla nas ważniejszy bo on pokazuje jak wielu terrorystów nie wyrkyliśmy
# powinno się dążyć do minimalizacji Type II error

0.25

In [ ]:
# Precision - ile obserwacji przewidzianych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
precision

0.75

In [ ]:
# Recall - jak wiele obserwacji z wszystkich pozytywnych sklasyfikowaliśmy jako pozytywne
recall = tp / (tp + fn)
recall

0.75

In [ ]:
# F1 score - średnia harmoniczna Precision i Recall

# Krzywa ROC



In [7]:
from sklearn.metrics import roc_curve

# fpr - false positive roc, tpr - true positive roc
#  pos_label = 1 => klasa "pozytywna" w naszym przypadku jest jest 1

fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
roc = pd.DataFrame({'fpr': fpr, 'tpr':tpr})
roc

,fpr,tpr
0,0.000000,0.00
1,0.285714,0.75
2,1.000000,1.00


In [10]:
def plot_roc_curve(y_pred, y_true):
#  Binary classification

  from sklearn.metrics import roc_curve
  fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

  fig = go.Figure(data=[go.Scatter(x=roc['fpr'] , y=roc['tpr'], line_color='red', name='ROC Curve'),
                       go.Scatter(x=[0,1], y=[0,1], mode='lines', line_dash='dash', line_color='navy')],
                  layout=go.Layout(xaxis_title='False Positive Rate',
                                   yaxis_title='True Psoitive Rate',
                                   title='ROC Curve',
                                   width=800))
  fig.show()

plot_roc_curve(y_pred, y_true)

#  zawsze trzeba stworzyć obiekt Figure
#  typ śladu, jaki chcemy stworzyć (Scatter)
# param Layout - jka nasz wykres ma wyglądać

Krzywa ROC to jest to czerwone. U nas jest 1 model więc jest 1 krzywa.  
Polega to na tym, że wybieramy ten model, który pod krzywą na największe pole.  
Dla modelu losowego widok to jest linia przerywana. Wszystko co jest ponad linie przerywaną działa lepiej niż model losowy. Staramy się dążyć aby Krzywa ROC była bliska wartości 1 i przez to miała jak największe pole. (Często też liczy się pola pod tym wykresem i na ich podstawie klasyfikuje modele).  

Krzywa ROC (Receiver Operating Characteristic) jest narzędziem używanym w analizie statystycznej, a szczególnie w dziedzinie diagnostyki medycznej, uczeniu maszynowym i inżynierii systemów detekcji. Służy do oceny jakości modeli klasyfikacyjnych, które przewidują wyniki binarne.

### Jak czytać Krzywą ROC:
- Oś X (1 - Specyficzność): Określa odsetek fałszywie pozytywnych wyników (błędów I rodzaju).
- Oś Y (Czułość): Określa odsetek prawdziwie pozytywnych wyników (trafności) (1 - błąd II rodzaju).

Im bliżej lewego górnego rogu znajduje się krzywa ROC, tym lepsza jest jakość modelu klasyfikacyjnego. Idealny model miałby krzywą przechodzącą przez punkt (0,1), co oznaczałoby 100% czułości i 100% specyficzności.

### Jak tworzyć Krzywą ROC:
1. **Uzyskaj wyniki klasyfikacji**: Najpierw musisz uzyskać wyniki klasyfikacji z modelu, wraz z prawdopodobieństwem przynależności do danej klasy.

2. **Oblicz punkty na krzywej ROC**: Następnie oblicz różne punkty na krzywej ROC, zmieniając próg klasyfikacji i obliczając czułość oraz specyficzność dla każdego progu.

3. **Narysuj krzywą ROC**: Na podstawie obliczonych punktów narysuj krzywą ROC, gdzie oś X to 1 - Specyficzność, a oś Y to Czułość.

W praktyce biblioteki do uczenia maszynowego, takie jak scikit-learn w Pythonie, oferują gotowe narzędzia do obliczania i rysowania krzywej ROC na podstawie wyników klasyfikacji.